In [5]:
# Librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import (accuracy_score,
                             precision_score,
                             recall_score)
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [6]:
project_path = "../bridge_project"
df_original = pd.read_csv(f"{project_path}/df1_original_cod.csv")
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   turns                  20058 non-null  int64
 1   white_rating           20058 non-null  int64
 2   black_rating           20058 non-null  int64
 3   opening_moves          20058 non-null  int64
 4   victory_status_cod     20058 non-null  int64
 5   winner_cod             20058 non-null  int64
 6   time_increment_cod     20058 non-null  int64
 7   opening_code_cod       20058 non-null  int64
 8   opening_fullname_cod   20058 non-null  int64
 9   opening_shortname_cod  20058 non-null  int64
 10  opening_variation_cod  20058 non-null  int64
 11  rated_cod              20058 non-null  int64
dtypes: int64(12)
memory usage: 1.8 MB


In [7]:
def eval_perform(Y,Yhat, train_or_test):
    """Evalúa el performance de cada modelo

    Args:
        Y (array): Variable objetivo original
        Yhat (array): Salida Y & "hat" (ŷ) que denota predicciones estimadas.
        train_or_test (string): "Entrenamiento" o "Prueba"
    """
    accu = accuracy_score(Y,Yhat)
    prec = precision_score(Y,Yhat,average='weighted')
    reca = recall_score(Y,Yhat,average='weighted')
    print(f"\nPerformance del modelo de {train_or_test}")
    print('Accu \t Prec \t Reca\n Eval \t %0.3f \t %0.3f \t %0.3f'%(accu,prec,reca))

# Df sin columnas altamente correlacionadas. 
Igual a df1 pero removiendo columnas correlacionadas:
- opening_fullname_cod se removió 
- black_raiting se removió
- opening_code_cod se removió

In [8]:
df = df_original.copy()
y_name = "winner_cod"
# X es el dataframe eliminando la variable de salida
X = df.drop(columns=[y_name])
# Y es un array unidimensional (ravel) de la variable de salida
Y = df[y_name].ravel()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.3)

# Regresión logística

In [9]:
from sklearn.linear_model import LogisticRegression
# Comparacion con regresion logistica
# Usar regularización L1 (Lasso)
reg_log = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)
reg_log.fit(X_train,Y_train)
# Salida Y & "hat" (ŷ) que denota predicciones estimadas.
Yhat_log_test = reg_log.predict(X_test)
Yhat_log_train = reg_log.predict(X_train)
eval_perform(Y_test,Yhat_log_test, "Prueba")
eval_perform(Y_train,Yhat_log_train, "Entrenamiento")


Performance del modelo de Prueba
Accu 	 Prec 	 Reca
 Eval 	 0.681 	 0.682 	 0.681

Performance del modelo de Entrenamiento
Accu 	 Prec 	 Reca
 Eval 	 0.663 	 0.664 	 0.663


# XGboost de ejemplo para comparar el performance (datos originales)

In [10]:
import xgboost as xgb
# Crear un clasificador XGBoost
xgboost = xgb.XGBClassifier()

# Entrenar el modelo en los datos de entrenamiento
xgboost.fit(X_train, Y_train)
Yhat_xgboost_test = xgboost.predict(X_test)
Yhat_xgboost_train = xgboost.predict(X_train)

eval_perform(Y_test,Yhat_xgboost_test, "Prueba")
eval_perform(Y_train,Yhat_xgboost_train, "Entrenamiento")


Performance del modelo de Prueba
Accu 	 Prec 	 Reca
 Eval 	 0.884 	 0.884 	 0.884

Performance del modelo de Entrenamiento
Accu 	 Prec 	 Reca
 Eval 	 0.954 	 0.954 	 0.954
